In [43]:
import numpy as np
from lightfm.datasets import fetch_movielens
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, reciprocal_rank

data = fetch_movielens()

In [44]:
print(repr(data['train']))
print(repr(data['test']))

<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 90570 stored elements in COOrdinate format>
<943x1682 sparse matrix of type '<class 'numpy.int32'>'
	with 9430 stored elements in COOrdinate format>


In [75]:
model = LightFM(loss='warp')
%time model.fit(data['train'], epochs=30, num_threads=2)

print("Train precision: %.2f" % precision_at_k(model, data['train'], k=5).mean())
print("Test precision: %.2f" % precision_at_k(model, data['test'], k=5).mean())

print("Train reciprocal rank: %.2f" % reciprocal_rank(model, data['train']).mean())
print("Test reciprocal rank: %.2f" % reciprocal_rank(model, data['test']).mean())

CPU times: user 2.82 s, sys: 0 ns, total: 2.82 s
Wall time: 1.45 s
Train precision: 0.67
Test precision: 0.12
Train reciprocal rank: 0.83
Test reciprocal rank: 0.28


In [74]:
model = LightFM(loss='bpr')
%time model.fit(data['train'], epochs=30, num_threads=2)

print("Train precision: %.2f" % precision_at_k(model, data['train'], k=5).mean())
print("Test precision: %.2f" % precision_at_k(model, data['test'], k=5).mean())

print("Train reciprocal rank: %.2f" % reciprocal_rank(model, data['train']).mean())
print("Test reciprocal rank: %.2f" % reciprocal_rank(model, data['test']).mean())

CPU times: user 2.62 s, sys: 0 ns, total: 2.62 s
Wall time: 1.36 s
Train precision: 0.67
Test precision: 0.10
Train reciprocal rank: 0.84
Test reciprocal rank: 0.24


In [6]:
def sample_recommendation(model, data, user_ids):


    n_users, n_items = data['train'].shape

    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]

        scores = model.predict(user_id, np.arange(n_items))
        top_items = data['item_labels'][np.argsort(-scores)]

        print("User %s" % user_id)
        print("     Known positives:")

        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")

        for x in top_items[:3]:
            print("        %s" % x)

sample_recommendation(model, data, [3, 25, 450])

User 3
     Known positives:
        Seven (Se7en) (1995)
        Indiana Jones and the Last Crusade (1989)
        Contact (1997)
     Recommended:
        Air Force One (1997)
        Contact (1997)
        Conspiracy Theory (1997)
User 25
     Known positives:
        Toy Story (1995)
        Twelve Monkeys (1995)
        Dead Man Walking (1995)
     Recommended:
        Fargo (1996)
        Trainspotting (1996)
        Scream (1996)
User 450
     Known positives:
        Kolya (1996)
        Devil's Own, The (1997)
        Contact (1997)
     Recommended:
        Air Force One (1997)
        Cop Land (1997)
        G.I. Jane (1997)
